In [26]:
%load_ext autoreload
%autoreload 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:

# Test loading NYC Taxi Zones shapefile
from bdproject.shapefile import get_gdf_shapefile

gdf = get_gdf_shapefile()
print("Shapefile loaded:", gdf.shape, "(rows, cols)")
gdf[["OBJECTID", "zone", "borough"]].head(10)


aksj
p /Users/tilen/pers/project/ext/out/external/taxi_zones/taxi_zones.shp
p.is_file() True
download True
Shapefile loaded: (263, 7) (rows, cols)


,OBJECTID,zone,borough
0,1,Newark Airport,EWR
1,2,Jamaica Bay,Queens
2,3,Allerton/Pelham Gardens,Bronx
3,4,Alphabet City,Manhattan
4,5,Arden Heights,Staten Island
5,6,Arrochar/Fort Wadsworth,Staten Island
6,7,Astoria,Queens
7,8,Astoria Park,Queens
8,9,Auburndale,Queens
9,10,Baisley Park,Queens


In [ ]:
# Inspect columns for original Yellow and FHV datasets (hardcoded path)
from pathlib import Path
import dask.dataframe as dd

EXT_SRC = Path("/Users/tilen/pers/project/ext/src")

paths_yellow = sorted(EXT_SRC.glob("yellow_tripdata_????-??.parquet"))
if not paths_yellow:
    paths_yellow = sorted((EXT_SRC / "yellow").glob("*.parquet"))

paths_fhvhv = sorted(EXT_SRC.glob("fhvhv_tripdata_????-??.parquet"))
if not paths_fhvhv:
    paths_fhvhv = sorted((EXT_SRC / "fhvhv").glob("*.parquet"))

print(f"Yellow: {len(paths_yellow)} files")
print(f"FHV:    {len(paths_fhvhv)} files")

if not paths_yellow:
    print("No Yellow files found under /Users/tilen/pers/project/ext/src or /yellow subdir")
if not paths_fhvhv:
    print("No FHV files found under /Users/tilen/pers/project/ext/src or /fhv subdir")

ddf_yellow = dd.read_parquet([str(p) for p in paths_yellow], engine="pyarrow", gather_statistics=False) if paths_yellow else None
ddf_fhvhv = dd.read_parquet([str(p) for p in paths_fhvhv], engine="pyarrow", gather_statistics=False) if paths_fhvhv else None

if ddf_yellow is not None:
    print("\nYELLOW columns (union across files):")
    print(sorted(map(str, ddf_yellow.columns)))
if ddf_fhvhv is not None:
    print("\nFHVHV columns (union across files):")
    print(sorted(map(str, ddf_fhvhv.columns)))



Yellow: 12 files
FHV:    12 files

YELLOW columns (union across files):
['Airport_fee', 'DOLocationID', 'PULocationID', 'RatecodeID', 'VendorID', 'congestion_surcharge', 'extra', 'fare_amount', 'improvement_surcharge', 'mta_tax', 'passenger_count', 'payment_type', 'store_and_fwd_flag', 'tip_amount', 'tolls_amount', 'total_amount', 'tpep_dropoff_datetime', 'tpep_pickup_datetime', 'trip_distance']

FHV columns (union across files):
['DOLocationID', 'PULocationID', 'access_a_ride_flag', 'airport_fee', 'base_passenger_fare', 'bcf', 'congestion_surcharge', 'dispatching_base_num', 'driver_pay', 'dropoff_datetime', 'hvfhs_license_num', 'on_scene_datetime', 'originating_base_num', 'pickup_datetime', 'request_datetime', 'sales_tax', 'shared_match_flag', 'shared_request_flag', 'tips', 'tolls', 'trip_miles', 'trip_time', 'wav_match_flag', 'wav_request_flag']


In [30]:
# Map LocationID to borough and show sample rows for Yellow and FHV
from bdproject.shapefile import get_gdf_shapefile
import re

# Build small lookup dict: OBJECTID (LocationID) -> borough
gdf = get_gdf_shapefile()
locid_to_borough = gdf[["OBJECTID", "borough"]].set_index("OBJECTID")["borough"].to_dict()


def _standardize_loc_cols(ddf):
    if ddf is None:
        return None
    cols = list(map(str, ddf.columns))
    norm = {re.sub(r"[^a-z0-9]", "", c.lower()): c for c in cols}
    rename = {}
    pu_src = norm.get("pulocationid")
    do_src = norm.get("dolocationid")
    if pu_src and pu_src != "pulocationid":
        rename[pu_src] = "pulocationid"
    if do_src and do_src != "dolocationid":
        rename[do_src] = "dolocationid"
    return ddf.rename(columns=rename) if rename else ddf


ddf_y = _standardize_loc_cols(ddf_yellow) if "ddf_yellow" in globals() else None
ddf_f = _standardize_loc_cols(ddf_fhvhv) if "ddf_fhvhv" in globals() else None


def _show_sample_with_borough(ddf, label):
    if ddf is None:
        print(f"{label}: no data")
        return
    # After standardization we should have canonical names
    needed = ["pulocationid", "dolocationid"]
    missing = [c for c in needed if c not in ddf.columns]
    if missing:
        print(f"{label}: missing expected columns {missing}")
        print("Available columns:", list(ddf.columns))
        return
    sample = ddf[needed].head(5, compute=True)
    sample["pu_borough"] = sample["pulocationid"].map(locid_to_borough)
    sample["do_borough"] = sample["dolocationid"].map(locid_to_borough)
    print(f"\n{label} sample with boroughs:")
    print(sample.to_string(index=False))


_show_sample_with_borough(ddf_y, "Yellow")
_show_sample_with_borough(ddf_f, "FHV")


aksj
p /Users/tilen/pers/project/ext/out/external/taxi_zones/taxi_zones.shp
p.is_file() True
download True

Yellow sample with boroughs:
 pulocationid  dolocationid pu_borough do_borough
          186            79  Manhattan  Manhattan
          140           236  Manhattan  Manhattan
          236            79  Manhattan  Manhattan
           79           211  Manhattan  Manhattan
          211           148  Manhattan  Manhattan

FHV sample with boroughs:
 pulocationid  dolocationid pu_borough do_borough
          161           158  Manhattan  Manhattan
          137            79  Manhattan  Manhattan
           79           186  Manhattan  Manhattan
          234           148  Manhattan  Manhattan
          148            97  Manhattan   Brooklyn


In [31]:
# Compute and print unique boroughs present in Yellow and FHV data
import dask.dataframe as dd


def _unique_boroughs(ddf):
    if ddf is None:
        return []
    cols = list(map(str, ddf.columns))
    if "pulocationid" not in cols or "dolocationid" not in cols:
        return []
    s = dd.concat([
        ddf["pulocationid"].map(locid_to_borough),
        ddf["dolocationid"].map(locid_to_borough),
    ])
    vals = s.dropna().drop_duplicates().compute().tolist()
    return sorted(set(vals))

boroughs_y = _unique_boroughs(ddf_y)
boroughs_f = _unique_boroughs(ddf_f)

print("Yellow boroughs:", boroughs_y)
print("FHV boroughs:", boroughs_f)
print("All boroughs:", sorted(set(boroughs_y) | set(boroughs_f)))


/Users/tilen/pers/project/.venv/lib/python3.13/site-packages/dask/dataframe/dask_expr/_collection.py:4208: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map function that you are using.
  Before: .map(func)
  After:  .map(func, meta=('pulocationid', 'object'))

  warnings.warn(meta_warning(meta, method="map"))
/Users/tilen/pers/project/.venv/lib/python3.13/site-packages/dask/dataframe/dask_expr/_collection.py:4208: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map function that you are using.
  Before: .map(fun

Yellow boroughs: ['Bronx', 'Brooklyn', 'EWR', 'Manhattan', 'Queens', 'Staten Island']
FHV boroughs: ['Bronx', 'Brooklyn', 'EWR', 'Manhattan', 'Queens', 'Staten Island']
All boroughs: ['Bronx', 'Brooklyn', 'EWR', 'Manhattan', 'Queens', 'Staten Island']


In [32]:
# Resolve top-10 pickup locids to zone and borough
import pandas as pd
from bdproject.shapefile import get_gdf_shapefile

# Top-N locids from consumer output
TOP_LOCIDS = [132, 138, 161, 230, 237, 79, 68, 236, 234, 162]

# Load shapefile
gdf = get_gdf_shapefile()

resolved = (
    pd.Series(TOP_LOCIDS, name="OBJECTID")
    .to_frame()
    .merge(gdf[["OBJECTID", "zone", "borough"]], how="left")
)
print(resolved.to_string(index=False))


aksj
p /Users/tilen/pers/project/ext/out/external/taxi_zones/taxi_zones.shp
p.is_file() True
download True
 OBJECTID                      zone   borough
      132               JFK Airport    Queens
      138         LaGuardia Airport    Queens
      161            Midtown Center Manhattan
      230 Times Sq/Theatre District Manhattan
      237     Upper East Side South Manhattan
       79              East Village Manhattan
       68              East Chelsea Manhattan
      236     Upper East Side North Manhattan
      234                  Union Sq Manhattan
      162              Midtown East Manhattan
